## Training a model from scratch ##

To train your own model from scratch, the ` Trainer ` class provides a simple implementation. Simply define your a custom dataset and a ` Trainer ` object, and call the ` Trainer ` 's ` fit() ` method.

## Writing a custom dataset ##

When creating a custom time series dataset class for use with Cents, the class must inherit from the provided `TimeSeriesDataset` base class. The `TimeSeriesDataset` class provides a robust and modular framework for handling wide-format time series data. Custom implementations only need to assign a name to `self.name` and implement the `_preprocess_data` method, which is an abstract method in the base class. This method should ensure that the data is available in a clean wide-format data frame, that has the structure outlined below.

### Responsibilities of `_preprocess_data`

- Preprocess raw input data into a DataFrame that satisfies the expected structure.
- Ensure time series columns contain arrays of the correct sequence length (`seq_len`).
- Add any additional columns, such as entity identifiers or context variables.

### Benefits of the Base Class

- **Normalization and Scaling:** Automatically handles standardization and min-max scaling.
- **context Variables:** Provides support for encoding and managing context variables.
- **Time Series Merging and Splitting:** Facilitates operations to merge multiple time series columns into a single multidimensional array and split them back when needed.
- **Data Transformation:** Includes functions for inverse transformations to revert normalized data to its original scale.

---

### Expected Input DataFrame Structure

The input to the `TimeSeriesDataset` class must adhere to the following structure:

| **Column Name**       | **Description**                                                                                     |
|------------------------|-----------------------------------------------------------------------------------------------------|
| `timeseries_col1`      | A column containing arrays of length `seq_len` (after preprocessing) representing the first dimension of the time series. |
| `timeseries_col2`      | A column containing arrays of length `seq_len` (after preprocessing) representing the second dimension of the time series.|
| `entity_column`        | A column containing unique identifiers for each entity (e.g., user, household, or device ID).       |
| `context_var1`    | An (optional) static or numeric context variable (e.g., categorical or continuous feature).                |
| `context_var2`    | Further (optional) static or numeric context variables.                                                    |

- The `time_series_column_names` parameter specifies which columns are part of the time series.
- The `entity_column_name` parameter identifies the column containing unique entity IDs.
- The `context_var_column_names` parameter defines additional context variables.

---

In [1]:
import pandas as pd
import numpy as np

from cents.datasets.timeseries_dataset import TimeSeriesDataset
from cents.trainer import Trainer

class CustomTimeSeriesDataset(TimeSeriesDataset):
    """
    A custom TimeSeriesDataset implementation for handling toy data.

    Input data structure:
    - time_series_col1, time_series_col2: Time series data with arrays of length seq_len.
    - entity_id: Unique identifier for each entity.
    - static_context: Categorical or numeric context variable.
    """
    def __init__(
        self,
        data: pd.DataFrame,
        seq_len: int = 16,
        normalize: bool = False,
        scale: bool = False,
    ):
        time_series_column_names = ["time_series_col1", "time_series_col2"]
        context_var_column_names = ["context_var"]

        super().__init__(
            data=data,
            time_series_column_names=time_series_column_names,
            context_var_column_names=context_var_column_names,
            seq_len=seq_len,
            normalize=normalize,
            scale=scale,

        )

    def _preprocess_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocesses the raw input data to ensure it conforms to the expected format.

        - Ensures time series columns contain arrays of length seq_len.
        - Ensures all required columns are present.

        Args:
            data (pd.DataFrame): The raw input data.

        Returns:
            pd.DataFrame: The preprocessed data.
        """
        required_columns = ["time_series_col1", "time_series_col2", "context_var"]
        for col in required_columns:
            if col not in data.columns:
                raise ValueError(f"Missing required column: {col}")

        for col in ["time_series_col1", "time_series_col2"]:
            data[col] = data[col].apply(
                lambda x: np.array(x).reshape(-1, 1) if isinstance(x, list) else x
            )
            data[col] = data[col].apply(
                lambda x: np.array(x) if isinstance(x, np.ndarray) else ValueError(f"Invalid data in {col}")
            )
        for col in ["time_series_col1", "time_series_col2"]:
            data[col] = data[col].apply(
                lambda x: x[:self.seq_len] if len(x) >= self.seq_len else ValueError(f"Sequence too short in {col}")
            )
        return data

Now that we have defined our dataset class, let's create some artificial timeseries columns and context variables which will comprise our dataset. Note that because we passed normalize=False, we are not using a parametric normalizer. If we set normalize=True, we will train a parametric normalizer with the default training config.

In [2]:
data = pd.DataFrame({
        "time_series_col1": [np.random.rand(16) for _ in range(100)],
        "time_series_col2": [np.random.rand(16) for _ in range(100)],
        "context_var": np.random.choice(["a", "b", "c"], size=100).tolist(),
    })

custom_dataset = CustomTimeSeriesDataset(data)
custom_dataset.data

,index,context_var,timeseries,is_frequency_rare,cluster,is_pattern_rare,is_rare
0,0,2,"[[0.846557892139889, 0.675851372726446], [0.06...",False,2,True,False
1,1,0,"[[0.3243788307509139, 0.5343991059414336], [0....",True,9,True,True
2,2,0,"[[0.9449073249670529, 0.8339985335984342], [0....",True,1,True,True
3,3,2,"[[0.7986038440807655, 0.5108029540184494], [0....",False,3,False,False
4,4,0,"[[0.3616056430625689, 0.7913010437773548], [0....",True,2,True,True
...,...,...,...,...,...,...,...
95,95,2,"[[0.12553835225328158, 0.008270644063553378], ...",False,9,True,False
96,96,1,"[[0.2647745296948498, 0.20852458676197916], [0...",False,0,True,False
97,97,0,"[[0.8153611215783628, 0.7881865609193799], [0....",True,3,False,False
98,98,1,"[[0.9726672872762161, 0.7746102461251492], [0....",False,4,True,False


We will now create a `Trainer` object by passing the name of the desired model and the dataset object. To start training, simply call `Trainer.fit()`.

In [3]:
trainer = Trainer(model_name="acgan", dataset=custom_dataset, overrides=["trainer.max_epochs=5", "trainer.strategy=auto"])
trainer.fit()

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | context_module | ContextModule     | 4.3 K  | train
1 | generator      | Generator         | 309 K  | train
2 | discriminator  | Discriminator     | 167 K  | train
3 | adv_loss       | BCEWithLogitsLoss | 0      | train
4 | aux_loss       | CrossEntropyLoss  | 0      | train
-------------------------------------------------------------
477 K     Trainable params
0         Non-trainable params
477 K     Total params
1.909     Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Once training is complete, we can create a data generator object that has access to the trained model and dataset information. To generate data, there is no need to load in a trained model. Simply define the context variables, and call the `DataGenerator` 's `generate()` method.

In [4]:
data_generator = trainer.get_data_generator()

In [5]:
data_generator.set_context(context_var=2)
generated_df = data_generator.generate(n=100)
generated_df

,context_var,timeseries
0,2,"[[0.35216045, 0.524036], [0.40860957, 0.526294..."
1,2,"[[0.39892834, 0.51737106], [0.41330722, 0.4979..."
2,2,"[[0.36629742, 0.54354775], [0.43322757, 0.5283..."
3,2,"[[0.36517504, 0.48562878], [0.4277414, 0.53091..."
4,2,"[[0.33892456, 0.5168193], [0.36130148, 0.55954..."
...,...,...
95,2,"[[0.36958477, 0.53076947], [0.41364932, 0.5436..."
96,2,"[[0.36272144, 0.5096975], [0.3763022, 0.571060..."
97,2,"[[0.36221316, 0.50699925], [0.40216798, 0.4204..."
98,2,"[[0.3906406, 0.5122451], [0.39646956, 0.502664..."


# Model Evaluation

The `Evaluator` class provides functionality to assess the quality of generated data compared to the original training data. It computes various metrics including:

- Distribution similarity between real and generated data
- Utility metrics
- Context-FID

To evaluate the trained model, we can use the `evaluate()` method of the `Trainer`. This method accepts:

- A dataset to evaluate against (typically the training dataset)
- Optional evaluation configuration parameters

The evaluation results provide insights into how well the model captures the underlying data patterns and maintains the relationship with context variables.

Let's run an evaluation on our trained model:

In [6]:
results = trainer.evaluate()
metadata = results["metadata"]
metrics = results["metrics"]

[CENTS] Using device: cuda for evaluation


[Cents] Training Discriminative Score Model:   0%|          | 0/2000 [00:00<?, ?it/s]

[Cents] Training Predictive Score Model:   0%|          | 0/5000 [00:00<?, ?it/s]

[Cents] Training Discriminative Score Model:   0%|          | 0/2000 [00:00<?, ?it/s]

[Cents] Training Predictive Score Model:   0%|          | 0/5000 [00:00<?, ?it/s]

In [7]:
metrics

{'DTW': {'mean': np.float64(1.4476364766831096),
  'std': np.float64(0.15302953610924613)},
 'MMD': {'mean': np.float64(0.022156818688420555),
  'std': np.float64(0.015161203706493065)},
 'Context_FID': np.float64(4.60251026124625),
 'Disc_Score': np.float64(0.5),
 'Pred_Score': 0.25806143090128897,
 'rare_subset': {'DTW': {'mean': np.float64(1.4136056858742925),
   'std': np.float64(0.18925874125581987)},
  'MMD': {'mean': np.float64(0.020573098136937147),
   'std': np.float64(0.013526630793298716)},
  'Context_FID': np.float64(4.420796978859233),
  'Disc_Score': np.float64(0.5),
  'Pred_Score': 0.25751564943272137},
 'disentanglement': {'MIG': {'mean': 0.0, 'context_var': np.float64(0.0)},
  'SAP': {'mean': 0.018952240377676066,
   'context_var': np.float64(0.018952240377676066)}}}